# Imports

In [22]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Tools.leica_tools import LeicaHandler
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from tqdm import tqdm
import tensorflow as tf
import keras
from keras.models import Model, load_model
import cv2
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

In [23]:
None or 1

1

# Dataset generation

In [3]:
def divide_frame(full_width, full_height, droplets, TILE=512, OVERLAP=0.2):

    MARGIN = int(TILE * OVERLAP)
    coords = []
    elements = []

    for y_min in range(0, full_height, TILE - MARGIN):
        for x_min in range(0, full_width, TILE - MARGIN):
            x_max = min(full_width, x_min + TILE)
            y_max = min(full_height, y_min + TILE)


            # select droplets that are fully contained
            subset = droplets.query(f'x_min >= {x_min} & x_max < {x_max} & y_min >= {y_min} & y_max < {y_max}').copy()

            if subset.empty:
                continue
            #transform coordinates of droplets bounding boxes into rel_xyxy
            subset['x_min'] = (subset['x_min'] - x_min)/(x_max-x_min)
            subset['x_max'] = (subset['x_max'] - x_min)/(x_max-x_min)
            subset['y_min'] = (subset['y_min'] - y_min)/(y_max-y_min)
            subset['y_max'] = (subset['y_max'] - y_min)/(y_max-y_min)

            coords.append((x_min, y_min, x_max, y_max))
            elements.append({
                'bboxes': subset[['x_min', 'y_min', 'x_max', 'y_max']].values.tolist(),
                'categories': subset['outlier'].astype(int).tolist(),
            })

    return coords, elements

In [464]:
EXP_ID = 'NKIP_FA_066'
OUTPUT_DIR = '/Users/fauberma/yolo_detection'
FNAME_PREFIX = EXP_ID
TILE_SIZE = 512
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)

exp = Experiment(EXP_ID)
droplet_df = exp.get_droplet_df()#.query('frameID < 3')
frame_df = exp.frame_df
annotations = {}
tiles = {}

for frameID, droplet_subset in tqdm(droplet_df.groupby('frameID'), desc="Processing frames"):
    frame, meta = exp.handler.get_frame(frameID)
    frame = Image.fromarray((frame[0]/ 256).astype(np.uint8)).convert("L")
    coords, elements = divide_frame(frame.width, frame.height, droplet_subset)

    for coord, element in zip(coords, elements):
        x_min, y_min, x_max, y_max = coord
        tileID = f"{FNAME_PREFIX}_{frameID}_{x_min}_{y_min}"

        image_path = os.path.join(OUTPUT_DIR, "images", f'{tileID}.png')
        if not os.path.exists(image_path):
            frame.crop((x_min, y_min, x_max, y_max)).save(image_path)

        annotations[tileID] = element
        tiles[tileID] = {'coord': coord, 'width': x_max-x_min, 'height':y_max-y_min}



# --- Save COCO-style annotations ---
with open(os.path.join(OUTPUT_DIR, f"{FNAME_PREFIX}_annotations.json"), "w") as f:
    json.dump({
        "images": tiles,
        "annotations": annotations,
    }, f, indent=2)

Processing frames: 100%|██████████| 25/25 [08:19<00:00, 19.99s/it]


# Training

In [16]:
def load_coco_dataset(data_dir, annotation_files, img_size=(512, 512), batch_size=8):
    data = {}
    for annotation_file in annotation_files:
        with open(os.path.join(data_dir, annotation_file), "r") as f:
            data.update(json.load(f)['annotations'])

    for key, val in data.items():
        bboxes = val['bboxes']
        for box in bboxes:
            assert(len(box) == 4)
    paths = [os.path.join(data_dir, 'images', f'{ID}.png') for ID in data.keys()]
    bboxes = tf.ragged.constant([data[ID]['bboxes'] for ID in data.keys()], dtype=tf.float32, ragged_rank=1, inner_shape=(4,))
    labels = tf.ragged.constant([data[ID]['categories'] for ID in data.keys()], dtype=tf.float32)

    def load_sample(path, bbox, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32)

        bounding_boxes = {
            "classes": label,
            "boxes": bbox,
        }
        return {"images": img, "bounding_boxes": bounding_boxes}

    ds = tf.data.Dataset.from_tensor_slices((paths, bboxes, labels))
    ds.shuffle(buffer_size=ds.cardinality())
    ds = ds.map(load_sample).ragged_batch(batch_size, drop_remainder=True)
    return ds

def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

In [17]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="rel_xyxy"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(512, 512), scale_factor=(0.4, 1.6), bounding_box_format="rel_xyxy"
        ),
    ]
)

In [18]:
ds = load_coco_dataset(
    data_dir="/Users/fauberma/yolo_detection",
    annotation_files=["NKIP_FA_082_annotations.json", "NKIP_FA_081_annotations.json","NKIP_FA_070_annotations.json","NKIP_FA_066_annotations.json"],
    img_size=(512, 512),
    batch_size=8,
)

In [19]:
val_ds = ds.take(200)
train_ds = ds.skip(200).take(1500)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)

In [20]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone"  # We will use yolov8 small backbone with coco weights
)

yolo = keras_cv.models.YOLOV8Detector(
    num_classes=2,
    bounding_box_format="rel_xyxy",
    backbone=backbone,
    fpn_depth=1,
)

optimizer = keras.optimizers.Adam(
    learning_rate=0.001,
    global_clipnorm=10.0,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)

In [21]:
yolo.fit(train_ds, steps_per_epoch=500, validation_data=val_ds, epochs=3,)

Epoch 1/3


2025-06-13 19:56:04.085529: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/RaggedSplit/assert_equal_3/Assert/AssertGuard/branch_executed/_733


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 787ms/step - box_loss: 1.4839 - class_loss: 7.3051 - loss: 8.7890

2025-06-13 20:03:15.489658: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/RaggedSplit/assert_equal_3/Assert/AssertGuard/branch_executed/_341


500/500 ━━━━━━━━━━━━━━━━━━━━ 477s 857ms/step - box_loss: 1.4833 - class_loss: 7.2957 - loss: 8.7790 - val_box_loss: 2.3946 - val_class_loss: 0.8728 - val_loss: 3.2674
Epoch 2/3
500/500 ━━━━━━━━━━━━━━━━━━━━ 414s 829ms/step - box_loss: 0.9471 - class_loss: 0.5957 - loss: 1.5428 - val_box_loss: 1.1263 - val_class_loss: 0.6033 - val_loss: 1.7296
Epoch 3/3
500/500 ━━━━━━━━━━━━━━━━━━━━ 422s 843ms/step - box_loss: 0.8363 - class_loss: 0.4605 - loss: 1.2968 - val_box_loss: 0.9168 - val_class_loss: 0.4339 - val_loss: 1.3507


In [516]:
yolo.save(os.path.join(os.getenv('MODEL_DIR'),'droplet_detection', 'yolo_v8_s_backbone_v3.keras'))